# REGLAS DE ASOCIACION | ALGORITMO APRIORI
> Autor: Elizon Frank Carcausto Mamani


In [ ]:
import numpy as np
import pandas as pd
import itertools as iter
from itertools import groupby, chain, combinations

## Generacion de ítemsets frecuentes

Esta funcion toma las playlist en formato de lista de listas y encuentra los conjuntos frecuentes en base al min_support.

In [ ]:
def get_frequent_itemsets(list_playlists, min_support):
  # Eliminarmos las canciones que se repiten en una las playlist
  list_playlists = [list(set(e)) for e in list_playlists]
  n = len(list_playlists)
  print(n)

  # Identificar los 1-items y contar sus indicencias en las transacciones
  list_all_playlists = list(iter.chain(*list_playlists))
  one_item = [(e, len(list(d))/n) for e, d in groupby(sorted(list_all_playlists))] # identificamos los n_items y contamos sus instancias 
  frec_one_item = [(e,s) for e,s in one_item if s >= min_support] # filtrar los items que son mayores o iguales al min_sopport
  # frec_one_item es una lista de tuplas que cumple ('1-item', support)

  # convertir a dataframe (por la gran cantidad de datos)
  df_playlists = pd.DataFrame(frec_one_item)
  df_playlists = df_playlists.rename(columns = {0:'items', 1:'support'})
  print('1_items: ', len(one_item))
  print(len(df_playlists),'\n' )

  # Encontramos los n_items frecuentes
  k = 2
  final = [] # conjunto final (lista de dataframe)
  df_set_frec = df_playlists
  # Empezamos el bucle desde 2_items
  while len(df_set_frec) > 1:
    n_items = merge_items(list(df_set_frec['items']), k)  # mesclamos los (n-1)_items para hallar los n_items
    print(str(k) + '_items: ', len(n_items))
    df_n_items = pd.DataFrame()
    df_n_items['items'] = n_items
    df_n_items['count'] = df_n_items['items'].apply(count_item, list_playlists = list_playlists) # contamos las instancias del n_item en las transacciones
    df_n_items['support'] = df_n_items['count']/n # Hallamos el soporte de cada n_item
    df_set_frec = df_n_items[df_n_items['support'] >= min_s] # filtramos los n_items con soporte mayor o igual al min_support
    print(len(df_set_frec),'\n')
    k += 1
    final.append(df_set_frec) 
  return pd.concat([final[i] for i in range(len(final))])

Esta funcion nos ayuda a contar las instancias de un n_item en el conjunto de transacciones

In [ ]:
# la logica de la funcion es, que filtremos los elementos en la que el 
# n_item se encuentra para luego solo retornar la longitud de la lista
def count_item(k_subset, list_playlists):
  if type(k_subset) is str:
    k_subset = [k_subset]
  count_inst = lambda seq: set(k_subset).issubset(set(seq)) # subfuncion verifica si un n_item esta entre las transacciones
  return len(list(filter(count_inst, list_playlists))) 

Esta funcion simplifica la tarea de mesclar los (n-1)items para hallar los n-items


In [ ]:
def merge_items(elem, n):
  if type(elem[0]) == str and n == 2:
    return sorted(list(combinations(elem, n))) # mesclar los 1-items
  else: # mesclar n-items
    tmp = list(combinations(elem, 2)) # combinamos todos los n_items en grupos de 2
    tmp2 = [tuple(set(iter.chain(*e))) for e in tmp if len(set(iter.chain(*e))) == n and e[0][0] == e[1][0]]  # filtramos los que tengan la longitd n
    items = []
    for e in tmp2:
      if e not in items:
        items.append(tuple(sorted(e))) # verificamos que los n-items no se repitan
    return items

## Generacion de reglas de asociacion

Esta funcion genera las reglas de asociacion en base a un umbral de confiancia y un lift.

In [ ]:
# La funcion recibe un dataframe de n_items mas frecuentes y la lista de transacciones
def generate_association_rules(frequent_itemsets, transactions ,confidence = 0, lift = 0):
  # Encontramos todas las posibles reglas en base al conjunto frecuente
  list_rules = []
  for i, row in frequent_itemsets.iterrows():
    df_tmp = pd.DataFrame()
    rules_tmp = rules(row['items'])
    df_tmp['izq'] = [e[0] for e in rules_tmp]
    df_tmp['der'] = [e[1] for e in rules_tmp]
    df_tmp['count'] = [row['count']] * len(rules_tmp)
    list_rules.append(df_tmp)
  df_rules = pd.concat([e for e in list_rules])

  # Hallamos los indicen de confianza y lift
  df_rules['count der'] = df_rules['der'].apply(count_item, list_playlists = transactions)
  df_rules['confidence'] = df_rules['count']/df_rules['count der']
  df_rules['support'] = df_rules['count']/len(transactions)
  df_rules['support der'] = df_rules['count der'] / len(transactions)
  df_rules['lift'] = df_rules['confidence'] / df_rules['support der']

  # filtramos las reglas que cumplan con los umbrales de confidence y lift
  df_rules_f_tmp = df_rules[df_rules['confidence'] >= confidence]
  df_rules_final = df_rules_f_tmp[df_rules_f_tmp['lift'] >= lift]
  return df_rules_final.drop(['count', 'count der', 'support der'], axis=1)

Esta funcion genera todas las reglas posibles en base a un n-item.

In [ ]:
def rules(item):
  if len(item) == 2:
    return [(item[0], item[1]), (item[1], item[0])] # reglas en base a 2-item
  else:
    tmp = []
    comple = lambda e, conj: tuple(set(conj) - set(e)) # funcion que halla el complemento de un subconjunto
    for i in range(len(item)-1, 0,-1):
      # filtrsmos la reglas que no se repitan
      tmp += [(e, comple(e, item)) for e in list(combinations(item, i)) if comple(e, item) not in tmp]
    return tmp

## Aplicado al dataset spotify.npy

Preparando los datos de spotify.npy

In [ ]:
playlists = np.load('spotify.npy', allow_pickle=True)
# convertir en una lista de listas
list_playlists = list(playlists.tolist().values())

Preparar los parametros: transacciones y  min_support.

In [ ]:
trans = list_playlists
min_s = 0.016 # min_support

Generar el conjunto de n-items frecuentes.

In [ ]:
set_f = get_frequent_itemsets(trans, min_s)
len(set_f)

Generar las reglas de asociasion 

In [ ]:
rules = generate_association_rules(set_f, trans, confidence=0.5, lift=1.2)

In [ ]:
rules

## Presentacion de las 10 mejores reglas de asociacion.

In [ ]:
rules_sort = rules.sort_values('confidence', ascending=False)
k = 1
for _, row in rules_sort.iterrows():
  print('Nro:', k)
  print('Regla: ', row['izq'] + ' ==> ' + row['der'])
  print('Soporte: ' , row['support'])
  print('Confianza: ', row['confidence'])
  print('Sustento: ', row['lift'])
  print('='*30)
  k += 1


## Comentar 4 reglas de asociacion

> ### Regla 1: HUMBLE. ==> DNA,  Soporte:  0.019,  Confianza:  0.8225, Sustento:  35.6065

La regla numero 1 se aprecia que el soporte es alto lo que nos dice que con respecto a otras reglas esta es la de mayor fraccion en las transacciones. Tambien es la que mejor confianza posee y por la misma razon tambien es el que mejor puntuacion presenta en el sustento.

Sin embargo ambos temas no comparten similitudes exactas de genero(Trap - pop), ritmo(bits energicos - bailable), o locatidad(Occidente - Oriente). La unica similitud que logre apreciar es que la tonalidad de los temas, ya que ambos destellan alegria. Esta es talvez una prueba de que hay cosas que los algoritmo pueden ver que nosotros no.

> ### Regla 4: Bad and Boujee (feat. Lil Uzi Vert) ==> Bounce Back, Soporte:  0.0169, Confianza:  0.5690, Sustento:  19.1590

Al realizar una busqueda de ambas cambiones se aprecia que ambas perteneces a un mismo genero y no solo eso sino que el estilo musical es muy pero muy similar, ambos acompañados de un bit tranquilo y con un trap que abordan la vida extravagante de un personaje. 



> ### Regla 8: XO TOUR Llif3 ==> Mask Off, Soporte: 0.0163, Confianza: 0.5158, Sustento:  16.3235

Como en los anteriores ejemplos se aprecia que ambos temas perteneces a un mismo genero, el Trap, tambien ambos temas hablan sobre una vida caotica, tambien cave resaltar que junto a la regla 4 estos presentan indicadores bastante parecidos, y no solo eso si no que el genero y ritmo tambien son notablemente similares, diferenciando en la tonalidad el cual es mas positiva que la regla 4.

> ### Regla 10: Mask Off ==> XO TOUR Llif3, Soporte: 0.0163, Confianza: 0.5046, Sustento:  15.6236

Pasa algo muy curioso con esta regla, ya que presentan los mismos temas que la Regla 8 solo que en orden inverso, tambien notamos que el indicador que mas difiere es el sustento. Una posible explicacion puede ser que el tema Mask off tuvo un alcance casi viral lo cual significaria que tiene un mayor soporte que el otro tema. Ya que el sustento de la Regla 10 esta fuertemente ligado al soporte del segundo tema, un tema menos viral que el primero.